In [1]:
#Imports

from contextlib import contextmanager

from OCC.Extend.DataExchange import write_step_file
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
from OCC.Core.BRepGProp import brepgprop 
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.GeomAbs import GeomAbs_Cylinder
from OCC.Core.gp import gp_Trsf, gp_Vec, gp_Pnt, gp_Dir, gp_Ax3
from OCC.Core.GProp import GProp_GProps
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.Poly import Poly_Triangulation
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_VERTEX
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import TopoDS_Vertex

from OCC.Extend.DataExchange import read_step_file
from OCC.Extend.TopologyUtils import TopologyExplorer

from sklearn.decomposition import PCA

from tqdm import tqdm

import hashlib
import numpy as np
import pandas as pd
import os
import tempfile
import uuid
import warnings


In [2]:
#load step file
@contextmanager
def suppress_user_warnings():
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        yield  # Run the code inside the 'with' block

# Load STEP file
step_reader = STEPControl_Reader()
# status = step_reader.ReadFile("C25001-1-0101.step")  # Replace with your file

status = step_reader.ReadFile("0444-1.step")  # Replace with your file

if status == IFSelect_RetDone:
    step_reader.TransferRoots()
    shape = step_reader.OneShape()
    print("STEP file loaded successfully.")
else:
    raise Exception("Failed to load STEP file.")

STEP file loaded successfully.


In [40]:
# ---------Test to remove

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def display_pca_alignment(shape, bbox_info, scale=1.0):
    # Start the viewer
    renderer = JupyterRenderer()
    renderer.DisplayShape(shape, update=True, transparency=True, opacity=0.1)

    # Draw PCA axes from center
    origin = gp_Pnt(*bbox_info["center"])
    directions = bbox_info["pca"].components_

    colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # X:Red, Y:Green, Z:Blue
    axis_labels = ['X', 'Y', 'Z']
    
    for i in range(3):
        dir_vec = directions[i]
        p1 = origin
        p2 = gp_Pnt(
            origin.X() + dir_vec[0] * 500 * scale,
            origin.Y() + dir_vec[1] * 500 * scale,
            origin.Z() + dir_vec[2] * 500 * scale,
        )
        edge = BRepBuilderAPI_MakeEdge(p1, p2).Edge()
        renderer.DisplayShape(edge, update=False)
        print(f"{axis_labels[i]} axis direction:", dir_vec)

    # renderer.View_Iso()
    # renderer.FitAll()
    return renderer


from sklearn.decomposition import PCA
from OCC.Core.BRep import BRep_Tool
from OCC.Extend.TopologyUtils import TopologyExplorer
import numpy as np

def get_vertices_array(shape):
    """Extract XYZ coordinates of all vertices in the shape."""
    verts = []
    topo = TopologyExplorer(shape)
    for v in topo.vertices():
        p = BRep_Tool.Pnt(v)
        verts.append([p.X(), p.Y(), p.Z()])
    return np.array(verts)

def get_aligned_bounding_box(shape):
    """Compute PCA-aligned bounding box and return dimensions and transformation info."""
    points = get_vertices_array(shape)
    if len(points) < 3:
        raise ValueError("Not enough points for PCA.")

    pca = PCA(n_components=3)
    aligned = pca.fit_transform(points)

    min_corner = np.min(aligned, axis=0)
    max_corner = np.max(aligned, axis=0)
    dims = max_corner - min_corner

    center_aligned = 0.5 * (max_corner + min_corner)
    center_global = pca.inverse_transform(center_aligned)

    return {
        "length": dims[0],
        "width": dims[1],
        "height": dims[2],
        "center": center_global,
        "pca": pca,
        "aligned_points": aligned,
        "bounding_box_min": min_corner,
        "bounding_box_max": max_corner,
    }


In [3]:
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Plane
from OCC.Core.gp import gp_Vec

def classify_plate_with_geometry(shape, bbox_dims, thickness_threshold=80, area_ratio_tol=0.02, min_face_area=100.0):
    dims = sorted(bbox_dims)  # Smallest first
    thickness = dims[0]
    
    if thickness > thickness_threshold:
        return None, 0.0  # Too thick to be a plate

    # Collect planar face areas and normals
    planar_faces = []
    face_explorer = TopExp_Explorer(shape, TopAbs_FACE)

    while face_explorer.More():
        face = face_explorer.Current()
        surf = BRepAdaptor_Surface(face)
        if surf.GetType() == GeomAbs_Plane:
            plane = surf.Plane()
            normal = plane.Axis().Direction()
            umin, umax = surf.FirstUParameter(), surf.LastUParameter()
            vmin, vmax = surf.FirstVParameter(), surf.LastVParameter()
            area = abs((umax - umin) * (vmax - vmin))
            if area >= min_face_area:
                planar_faces.append((area, normal))
        face_explorer.Next()

    # Check for pair of large, nearly equal, opposite planar faces
    for i, (area1, normal1) in enumerate(planar_faces):
        for j, (area2, normal2) in enumerate(planar_faces):
            if i >= j:
                continue
            area_diff = abs(area1 - area2) / max(area1, area2)
            dot = normal1.Dot(normal2)
            if area_diff < area_ratio_tol and abs(dot + 1) < 0.1:
                return "IfcPlate", 0.95  # Confident match

    return None, 0.0  # Not a plate



In [4]:
def has_cylindrical_surface(shape, min_radius=2.0):
    face_explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while face_explorer.More():
        face = face_explorer.Current()
        surf = BRepAdaptor_Surface(face)
        if surf.GetType() == GeomAbs_Cylinder:
            cyl = surf.Cylinder()
            if cyl.Radius() > min_radius:
                return True
        face_explorer.Next()
    return False

In [5]:
def classify_nut_from_shape(shape, bbox_dims, thickness_threshold=100, side_count=6, angle_tol=10, area_tol=0.15):
    dims = sorted(bbox_dims)
    thickness = dims[0]
    
    if thickness > thickness_threshold:
        return None, 0.0

    # Extract face normals and areas
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    normals = []
    areas = []
    
    while explorer.More():
        face = explorer.Current()
        surf = BRepAdaptor_Surface(face)
        
        if surf.GetType() == GeomAbs_Plane:
            umin, umax, vmin, vmax = surf.FirstUParameter(), surf.LastUParameter(), surf.FirstVParameter(), surf.LastVParameter()
            try:
                u = (umin + umax) / 2
                v = (vmin + vmax) / 2
                d1u, d1v = gp_Vec(), gp_Vec()
                surf.D1(u, v, gp_Pnt(), d1u, d1v)
                normal = d1u.Crossed(d1v)
                if normal.Magnitude() > 0:
                    normals.append(np.array([normal.X(), normal.Y(), normal.Z()]))
                    area = BRep_Tool.Surface(face).Surface().Area()
                    areas.append(area)
            except:
                pass
        
        explorer.Next()
    
    # Group similar normals (within angular tolerance)
    grouped = []
    for n in normals:
        matched = False
        for group in grouped:
            angle = np.degrees(np.arccos(np.clip(np.dot(n, group[0]) / (np.linalg.norm(n) * np.linalg.norm(group[0])), -1.0, 1.0)))
            if angle < angle_tol:
                group.append(n)
                matched = True
                break
        if not matched:
            grouped.append([n])

    num_flat_sides = len(grouped)

    if num_flat_sides == side_count:
        return "IfcMechanicalFastener:Nut", 0.95
    
    return None, 0.0


In [6]:
def classify_bolt_from_bbox(shape, bbox_dims, circular_tol=0.05, thickness_ratio=0.8):
    dims = sorted(bbox_dims)  # [length, width, height]
    length, width, height = dims
    cross_section = sorted([width, height])
    width, depth = cross_section

    # Criteria 1: Roughly circular cross-section
    if abs(width - depth) / max(width, depth) > circular_tol:
        return None, 0.0

    # Criteria 2: Short in length
    if length < max(width, depth) * thickness_ratio:
        return "IfcFastener", 0.9  # Likely a bolt

    return None, 0.0

In [7]:
def classify_bolt(shape, bbox_dims):
    type_guess, confidence = classify_bolt_from_bbox(shape, bbox_dims)
    if type_guess and has_cylindrical_surface(shape):
        return type_guess, confidence
    return None, 0.0

In [8]:
def classify_profile(row):
    width = row["Width"]
    height = row["Height"]
    depth = row["Depth"]

    shape = row["AlignedShape"]
    bbox = (row["Width"], row["Height"], row["Depth"])

    bbox_dims = (row["BBoxX"], row["BBoxY"], row["BBoxZ"])
    
    plate_type, confidence = classify_plate_with_geometry(row["AlignedShape"], bbox_dims)
    
    if plate_type:
        return {"ProfileType": plate_type, "Confidence": confidence}

    # Check for Bolt
    bolt_type, confidence = classify_bolt_from_bbox(shape, bbox)
    if bolt_type and has_cylindrical_surface(shape):
        return bolt_type 

     # Check for Nut
    nut_type, confidence = classify_nut_from_shape(shape, bbox)
    if nut_type:
        return {"ProfileType": nut_type, "Confidence": confidence}
    
    # Beam: One large dimension (width/height) and small depth
    if width > height and width > depth:
        return "Beam"
    
    # Column: Large height and width similar to depth
    elif height > width and height > depth:
        return "Column"
    
    # Angle: Equal width and height
    elif abs(width - height) < 0.1 * width:  # Adjust threshold as needed
        return "Angle"
    
    # Default: Other profiles
    return "Other"


In [9]:
def align_shape_to_origin_and_axes(shape):
    """Align shape to local axes via PCA and recenter it to (0,0,0)."""
    points = sample_points_from_shape(shape)
    if len(points) < 3:
        return shape, gp_Trsf()  # Identity transform if not enough points

    # Compute PCA
    centroid = np.mean(points, axis=0)
    rotation_matrix = compute_pca_alignment_matrix(points)

    # Apply alignment transform (axes alignment only)
    align_trsf = create_alignment_transform(rotation_matrix, centroid)
    aligned_shape = BRepBuilderAPI_Transform(shape, align_trsf, True).Shape()

    # Now compute the centroid of the aligned shape
    props = GProp_GProps()
    brepgprop.VolumeProperties(aligned_shape, props)
    new_centroid = props.CentreOfMass()

    # Create translation transform to recenter at (0, 0, 0)
    recenter_trsf = gp_Trsf()
    recenter_trsf.SetTranslation(new_centroid, gp_Pnt(0, 0, 0))
    final_shape = BRepBuilderAPI_Transform(aligned_shape, recenter_trsf, True).Shape()

    # Combine both transforms: final = recenter * align
    align_trsf.Multiply(recenter_trsf)

    return final_shape, align_trsf


In [10]:
def sample_points_from_shape(shape, deflection=0.5):
    """
    Triangulates the shape (if needed) and extracts vertex points from its faces.
    This avoids using hole geometry by just considering outer triangulations.
    """
    BRepMesh_IncrementalMesh(shape, deflection)
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    points = []

    while explorer.More():
        face = explorer.Current()
        triangulation = BRep_Tool.Triangulation(face, TopLoc_Location())
        if triangulation is not None:
            for i in range(triangulation.NbNodes()):
                pnt = triangulation.Node(i + 1)
                points.append([pnt.X(), pnt.Y(), pnt.Z()])
        explorer.Next()

    return np.array(points)

In [11]:
def create_alignment_transform(rotation_matrix, centroid):
    """Create a gp_Trsf transformation to align the shape axes and move centroid to origin."""
    vx = gp_Dir(*rotation_matrix[:, 0])
    vy = gp_Dir(*rotation_matrix[:, 1])
    vz = gp_Dir(*rotation_matrix[:, 2])

    local_axis = gp_Ax3(gp_Pnt(*centroid), vz, vx)

    trsf = gp_Trsf()
    trsf.SetTransformation(gp_Ax3(), local_axis)
    return trsf


In [12]:
def compute_pca_alignment_matrix(points):
    """Perform PCA on the point cloud to get principal axes and return rotation matrix."""
    pca = PCA(n_components=3)
    pca.fit(points - np.mean(points, axis=0))
    rotation_matrix = pca.components_.T  # Each column is a principal axis
    return rotation_matrix

In [13]:
def shape_to_hash(shape, precision=4):
    """
    Generates a stable hash for a TopoDS_Shape by extracting and sorting its vertex coordinates.
    """
    vertex_coords = []
    explorer = TopExp_Explorer(shape, TopAbs_VERTEX)

    while explorer.More():
        vertex = explorer.Current()  # already a TopoDS_Vertex
        pnt = BRep_Tool.Pnt(vertex)
        coords = (
            round(pnt.X(), precision),
            round(pnt.Y(), precision),
            round(pnt.Z(), precision)
        )
        vertex_coords.append(coords)
        explorer.Next()

    vertex_coords.sort()
    coords_str = str(vertex_coords).encode('utf-8')
    return hashlib.md5(coords_str).hexdigest()



In [45]:
# df_solid generation - dataframe of solids from full model

explorer = TopologyExplorer(shape)
             
# Prepare data list
data = []

for index, solid in enumerate(tqdm(list(explorer.solids()), desc="Traversing Model")):

    # --------Test Start


    renderer = JupyterRenderer()
    renderer.DisplayShape(solid, update=True)
    renderer.display.FitAll()
    print(solid.IsNull())
    
    bbox_info = get_aligned_bounding_box(solid)

    print("Length (mm):", bbox_info["length"])
    print("Width (mm):", bbox_info["width"])
    print("Height (mm):", bbox_info["height"])
    print("Centroid:", bbox_info["center"])

    display_pca_alignment(solid, bbox_info, scale=1.0)

    # ------Test End
   
    # Volume
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    volume = props.Mass()

    # Surface area
    surf_props = GProp_GProps()
    brepgprop.SurfaceProperties(solid, surf_props)
    area = surf_props.Mass()

    # Centroid
    centroid = props.CentreOfMass()
    centroid_coords = (centroid.X(), centroid.Y(), centroid.Z())

    #Align the solid to local origin and axis
    aligned_solid, transform_data = align_shape_to_origin_and_axes(solid)

    # Debug transform result
    # print(f"Item {index}")
    # print(f"  Original centroid: {centroid_coords}")
    # print(f"  Translation: {translation.X():.2f}, {translation.Y():.2f}, {translation.Z():.2f}")
    # print(f"  Rotation quaternion: {rotation_quaternion}")
    # print(f"  Aligned centroid: ({aligned_centroid.X():.2f}, {aligned_centroid.Y():.2f}, {aligned_centroid.Z():.2f})")
    # print(f"  BBox: {bbox_size}")
    # print("=" * 60)
    
    # Store transform data in a more accessible way
    translation = transform_data.TranslationPart()  # Get translation part as gp_Vec
    
    # Retrieve the rotation quaternion from the transformation matrix
    rotation = transform_data.GetRotation()  # This gets the rotation quaternion
    
    # Extract quaternion components (w, x, y, z)
    rotation_quaternion = (rotation.W(), rotation.X(), rotation.Y(), rotation.Z())

    # Recalculate centroid AFTER alignment
    props_aligned = GProp_GProps()
    brepgprop.VolumeProperties(aligned_solid, props_aligned)
    aligned_centroid = props_aligned.CentreOfMass()
    aligned_centroid_coords = (aligned_centroid.X(), aligned_centroid.Y(), aligned_centroid.Z())

    #Hash shape
    shape_hash = shape_to_hash(aligned_solid)  
    
    # Bounding box
    bbox = Bnd_Box()
    brepbndlib.Add(aligned_solid, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    bbox_size = (xmax - xmin, ymax - ymin, zmax - zmin)
    
    width = xmax - xmin
    height = ymax - ymin
    depth = zmax - zmin

    # Corrected length calculation
    length = max(width, height, depth)

    data.append({
        "ID": index,
        "Hash": shape_hash,
        "Width": round(width,2),
        "Height": round(height,2),
        "Depth": round(depth,2),
        "Volume": volume,
        "SurfaceArea": area,
        "CentroidX": aligned_centroid.X(),
        "CentroidY": aligned_centroid.Y(),
        "CentroidZ": aligned_centroid.Z(),
        "Rotation": rotation_quaternion,
        "Translation": (translation.X(), translation.Y(), translation.Z()),
        "AlignedShape":aligned_solid,
        "BBoxX": xmax - xmin,
        "BBoxY": ymax - ymin,
        "BBoxZ": zmax - zmin,
        "ApproxLength": length
    })

# Convert to DataFrame
df_solids = pd.DataFrame(data)

# Apply classification logic
df_solids["ProfileType"] = df_solids.apply(classify_profile, axis=1)



Traversing Model:   0%|               | 0/1 [00:00<?, ?it/s]

Traversing Model:   0%|               | 0/1 [00:00<?, ?it/s]


AttributeError: 'JupyterRenderer' object has no attribute 'display'

In [15]:
import pandas as pd

# Group by hash, aggregating by taking the mean for numerical columns (except for Hash)
df_aligned = df_solids.groupby('Hash').agg(
    {
        'Width': 'mean',
        'Height': 'mean',
        'Depth': 'mean',
        'Volume': 'sum',  # Or mean, depending on your desired aggregation
        'SurfaceArea': 'sum',  # Same here, you can sum or average based on use case
        'CentroidX': 'mean',
        'CentroidY': 'mean',
        'CentroidZ': 'mean',
        'Rotation': 'first',  # Assuming rotation is the same for identical items
        'Translation': 'first',  # Same as Rotation
        'AlignedShape': 'first',  # Or aggregate the shape objects in a way that works for you
        'BBoxX': 'mean',
        'BBoxY': 'mean',
        'BBoxZ': 'mean',
        'ApproxLength': 'mean',
        'ProfileType': 'first'  # Assuming profile type is the same for identical items
    }).reset_index()

# Create a quantity column to reflect the number of merged items
df_aligned['Quantity'] = df_solids.groupby('Hash')['Hash'].transform('count')

# Now `df_aligned` contains merged items with a count of identical items
# print(df_aligned)


In [16]:
# df_aligned

In [17]:
import pyvista as pv
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Extend.DataExchange import write_stl_file
import os
import shutil
from IPython.display import HTML
from tqdm import tqdm

# Define subdirectory
preview_dir = "previews/aligned_solids"
if os.path.exists(preview_dir):
    shutil.rmtree(preview_dir)
os.makedirs(preview_dir, exist_ok=True)

preview_paths = []

for index, row in tqdm(df_aligned.iterrows(), total=len(df_aligned), desc="Rendering aligned previews"):
    solid = row["AlignedShape"]
    # Mesh the shape if needed
    mesh = BRepMesh_IncrementalMesh(solid, 0.5)
    mesh.Perform()

    # Save STL temporarily
    stl_path = os.path.join(preview_dir, f"aligned_solid_{index+500}.stl")
    write_stl_file(solid, stl_path)

    # Load with PyVista
    pv_mesh = pv.read(stl_path)

    # Render preview
    plotter = pv.Plotter(off_screen=True, lighting=None)
    plotter.add_mesh(
        pv_mesh,
        silhouette={'feature_angle':20, 'color':"grey"},
        color="white", 
        show_edges=False, 
        edge_color="blue", 
        smooth_shading=False, 
        style="surface", 
        lighting=True
    )

    # Add origin sphere (optional, you can keep this if you want)
    # origin_sphere = pv.Sphere(radius=(min(width, height, depth)*1.5), center=(0, 0, 0))
    # plotter.add_mesh(origin_sphere, color='red', specular=1.0, smooth_shading=True)
    
    plotter.set_background("white")
    # plotter.camera_position = 'iso'
    plotter.camera_position = 'xy'
    
    img_path = os.path.join(preview_dir, f"aligned_solid_{index+500}.png")
    plotter.screenshot(img_path)
    plotter.close()

    preview_paths.append(img_path)

# Add image paths to DataFrame (for aligned items)
df_aligned["AlignedPreview"] = preview_paths

# Display DataFrame with images (in Jupyter)
def image_formatter(path):
    return f'<a href="{path}" target="_blank"><img src="{path}" width="200"/></a>'

HTML(df_aligned.to_html(escape=False, formatters={"AlignedPreview": image_formatter}))


Rendering aligned previews: 100%|█| 1/1 [00:03<00:00,  3.70


,Hash,Width,Height,Depth,Volume,SurfaceArea,CentroidX,CentroidY,CentroidZ,Rotation,Translation,AlignedShape,BBoxX,BBoxY,BBoxZ,ApproxLength,ProfileType,Quantity,AlignedPreview
0,d7278c33b4c6226aad0643322576ab66,2282.47,203.2,233.4,1.335289e+07,2.727574e+06,-5.356563e-14,2.221201e-14,1.115951e-14,"(2.9367933939848695e-16, 0.702467273550308, 2.9754595834786273e-16, 0.7117160456184732)","(-1056.0700834675347, 1.3675922872956692e-13, -98.10558047866692)",,2282.467947,203.2,233.403751,2282.467947,Beam,1,


In [18]:
# Start of NC1 programming

from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Cylinder
from OCC.Core.gp import gp_Cylinder, gp_Pnt, gp_Dir
import numpy as np

def extract_cylindrical_holes(shape: TopoDS_Shape, min_radius=2.0, max_radius=100.0):
    """
    Extract cylindrical hole data from a TopoDS_Shape.
    
    Returns a list of dicts with hole parameters.
    """
    holes = []

    face_explorer = TopExp_Explorer(shape, TopAbs_FACE)
    while face_explorer.More():
        face = face_explorer.Current()
        surface = BRepAdaptor_Surface(face)

        if surface.GetType() == GeomAbs_Cylinder:
            cyl: gp_Cylinder = surface.Cylinder()
            radius = cyl.Radius()
            
            # Filter radius range (to avoid false positives)
            if min_radius <= radius <= max_radius:
                axis = cyl.Axis()
                location = axis.Location()
                direction = axis.Direction()

                hole_data = {
                    "radius": radius,
                    "center": (location.X(), location.Y(), location.Z()),
                    "direction": (direction.X(), direction.Y(), direction.Z()),
                }
                holes.append(hole_data)

        face_explorer.Next()
    
    return holes


In [19]:
from tqdm import tqdm

# Detect holes only for non-plates and known types
df_aligned["DrilledHoles"] = [
    extract_cylindrical_holes(row["AlignedShape"])
    if row["ProfileType"] not in ["Plate", "Unknown"]
    else []
    for _, row in tqdm(df_aligned.iterrows(), total=len(df_aligned), desc="Detecting holes")
]



Detecting holes: 100%|██████| 1/1 [00:00<00:00, 345.55it/s]


In [20]:
df_aligned

,Hash,Width,Height,Depth,Volume,SurfaceArea,CentroidX,CentroidY,CentroidZ,Rotation,Translation,AlignedShape,BBoxX,BBoxY,BBoxZ,ApproxLength,ProfileType,Quantity,AlignedPreview,DrilledHoles
0,d7278c33b4c6226aad0643322576ab66,2282.47,203.2,233.4,1.335289e+07,2.727574e+06,-5.356563e-14,2.221201e-14,1.115951e-14,"(2.9367933939848695e-16, 0.702467273550308, 2....","(-1056.0700834675347, 1.3675922872956692e-13, ...",<class 'TopoDS_Solid'>,2282.467947,203.2,233.403751,2282.467947,Beam,1,previews/aligned_solids\aligned_solid_500.png,"[{'radius': 6.0, 'center': (889.4479500170133,..."


CREATING NC1 / DSTV

In [21]:
def write_dstv_file(row, output_dir="dstv_output"):
    import os

    os.makedirs(output_dir, exist_ok=True)
    
    hashid = row["Hash"]
    profile = row["ProfileType"]
    length = row["Width"]
    holes = row["DrilledHoles"]

    dstv_lines = []

    # Header
    dstv_lines.append(f"ST\n")  # DSTV file start
    dstv_lines.append(f"BO {hashid} {profile} S355JR {length:.2f} 0 0 0 0 0 0\n")

    # Holes (PU = drilled holes)
    for hole in holes:
        print(hole)
        center = hole['center']
        diameter = hole['radius'] * 2
        x = center[0]
        y = center[1]
        side = hole['side']  # A (top), V (bottom), etc.
        dstv_lines.append(f"PU {side} {x:.2f} {y:.2f} {dia:.2f}\n")

    # End
    dstv_lines.append("EN\n")

    # Write to file
    filepath = os.path.join(output_dir, f"{hashid}.nc1")
    with open(filepath, "w") as f:
        f.writelines(dstv_lines)

    return filepath


sample_row = df_aligned.iloc[332]
dstv_path = write_dstv_file(sample_row)
print(f"DSTV file written: {dstv_path}")

IndexError: single positional indexer is out-of-bounds